 # Marketing Campaign Creator-Critic Orchestration System
 
 ## Overview
 This notebook demonstrates the **creator-critic pattern** with **iterative refinement**. A creator agent generates marketing campaigns, a critic agent reviews them, and the process repeats until the campaign meets quality standards or reaches maximum iterations.

  <div align="center">
<img src="lesson_5.png" alt="Alt text" width="500"/>
</div>
 
 ### Key Concepts Covered:
 1. **Sequential Orchestration**: Agents work in a defined sequence
 2. **Iterative Refinement**: Feedback loop between creator and critic
 3. **Quality Gates**: Critic approves or requests revisions
 4. **InProcessRuntime**: Runtime environment for agent execution
 5. **Stateful Loops**: Tracking approval status across iterations
 6. **Callback Functions**: Monitoring agent responses in real-time

 ## 1. Setup and Configuration

In [1]:
import os
from dotenv import load_dotenv

from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.agents import ChatCompletionAgent, SequentialOrchestration
from semantic_kernel.contents import ChatMessageContent
from semantic_kernel.agents.runtime import InProcessRuntime

# Load environment variables
load_dotenv()
api_key = os.getenv("AZURE_OPENAI_KEY")
url = os.getenv("URL")
api_version = "2024-12-01-preview"

 ## 2. Initialize Kernel and Service

In [2]:
kernel = Kernel()

# Configure Azure OpenAI service
chat_service = AzureChatCompletion(
    deployment_name="none",
    api_key=api_key,
    base_url=url,
    api_version=api_version,
)

kernel.add_service(chat_service)

 ## 3. Define Creator and Critic Agents
 
 The **Creator Agent** generates marketing campaigns.
 The **Critic Agent** reviews and provides feedback.

 ### 3.1 Creator Agent (Campaign Generator)

In [3]:
creator_agent = ChatCompletionAgent(
    name="CampaignCreatorAgent",
    service=chat_service,
    instructions=(
        "Role: You are a creative marketing specialist who drafts comprehensive social media campaigns."
        "\n\nObjectives:"
        "\n• Write the campaign as a sequence of clear, numbered posts/actions"
        "\n• Use engaging, brand-appropriate language suitable for social media"
        "\n• Ensure posts follow a logical narrative arc (awareness → engagement → conversion → retention)"
        "\n• Incorporate marketing best practices (CTAs, hashtags, engagement hooks)"
        "\n• If supplied a REVISE label and feedback from the Critic Agent, update the campaign accordingly"
        "\n\nWorkflow:"
        "\n1. Check if there is a REVISE label in the input task"
        "\n2. If REVISE is present, carefully review the Critic's feedback and update the campaign to address all points"
        "\n3. If no REVISE label, create a new campaign from scratch based on the brief"
        "\n\nFormat each campaign post as:"
        "\nPost #: [Platform] - [Content Type]"
        "\nContent: [Post text]"
        "\nHashtags: [Relevant hashtags]"
        "\nCTA: [Call to action]"
        "\nTiming: [When to post]"
    ),
)

 ### 3.2 Critic Agent (Campaign Reviewer)

In [4]:
critic_agent = ChatCompletionAgent(
    name="CampaignCriticAgent",
    service=chat_service,
    instructions=(
        "Role: Review the marketing campaign against best practices, brand guidelines, and effectiveness criteria."
        "\n\nObjectives:"
        "\n• The campaign should include only a numbered list of posts with their details"
        "\n• Campaign should be formatted as numbered posts with clear structure"
        "\n• Example format:"
        "\n  Post 1: [Platform] - [Content Type]"
        "\n  Content: [Text]"
        "\n  Hashtags: [Tags]"
        "\n  CTA: [Action]"
        "\n  Timing: [Schedule]"
        "\n\nOnly provide feedback on:"
        "\n• Posts being numbered and properly structured"
        "\n• Each post must include: Platform, Content Type, Content, Hashtags, CTA, Timing"
        "\n• Content should be engaging and on-brand"
        "\n• Campaign should follow a logical progression (awareness → engagement → conversion)"
        "\n• CTAs should be clear and actionable"
        "\n• Hashtags should be relevant and trending"
        "\n\nWorkflow:"
        "\n1. If the campaign is satisfactory, respond with 'APPROVED' and show the final campaign"
        "\n2. Otherwise, provide detailed feedback for revisions and add label 'REVISE'"
        "\n3. Include the original campaign text in your response for context, labeled as 'Original Campaign:'"
    ),
)

 ## 4. Define Sequential Orchestration
 
 **SequentialOrchestration** ensures agents execute in order:
 1. Creator generates campaign
 2. Critic reviews and provides feedback
 3. Loop continues until approved or max iterations reached

 ### 4.1 Agent Response Callback

In [5]:
def agent_response_callback(message: ChatMessageContent) -> None:
    """
    Callback function invoked after each agent responds.
    Useful for monitoring, logging, and debugging.
    
    Args:
        message: The ChatMessageContent from the agent
    """
    print(f"\nAgent Response: {message.name}")
    print("-" * 70)

 ### 4.2 Create Sequential Orchestration

In [6]:
sequential_orchestration = SequentialOrchestration(
    members=[creator_agent, critic_agent],
    agent_response_callback=agent_response_callback,
)

 ## 5. Define Initial Campaign Brief

In [7]:
initial_task = """
Create a 5-post social media campaign for launching a new eco-friendly water bottle product.

Product Details:
- Name: HydroEco Pro
- Features: 100% recycled materials, keeps drinks cold for 24 hours, leak-proof, dishwasher safe
- Target Audience: Environmentally conscious millennials and Gen Z (ages 18-35)
- Price Point: $29.99
- Brand Voice: Friendly, inspirational, eco-focused

Campaign Goals:
- Build awareness for product launch
- Drive pre-orders
- Grow social media following
- Establish eco-friendly brand identity
"""

 ## 6. Main Orchestration Loop
 
 This implements the **iterative refinement pattern**:
 - Creator drafts → Critic reviews → Creator revises → Critic re-reviews
 - Loop continues until APPROVED or max iterations (10)

In [8]:
async def main():
    """
    Orchestrates the creator-critic feedback loop for campaign development.
    
    Process:
    1. Initialize runtime environment
    2. Submit task to sequential orchestration
    3. Creator generates campaign
    4. Critic reviews campaign
    5. If not approved, creator revises based on feedback
    6. Repeat until approved or max iterations reached
    """
    
    print("=" * 70)
    print("MARKETING CAMPAIGN CREATOR-CRITIC ORCHESTRATION SYSTEM")
    print("=" * 70)
    
    # Initialize runtime
    runtime = InProcessRuntime()
    runtime.start()
    
    # State management
    approved = False
    iteration = 1
    max_iterations = 10
    current_task = initial_task
    
    # Iterative refinement loop
    while not approved and iteration <= max_iterations:
        print(f"\n{'='*70}")
        print(f"ITERATION {iteration}")
        print("=" * 70)
        
        print("\n" + "-" * 70)
        print("Current Task:")
        print("-" * 70)
        print(current_task)
        print("-" * 70)
        
        # Execute sequential orchestration (Creator → Critic)
        orchestration_result = await sequential_orchestration.invoke(
            task=current_task,
            runtime=runtime,
        )
        
        # Get final report from orchestration
        final_report = await orchestration_result.get()
        
        print("\n" + "-" * 70)
        print("Final Report:")
        print("-" * 70)
        print(final_report.content[:500] + "..." if len(final_report.content) > 500 else final_report.content)
        print("-" * 70)
        
        # Update task with critic's feedback for next iteration
        current_task = final_report.content
        
        # Check approval status
        if "APPROVED" in final_report.content.upper():
            approved = True
            print("\n" + "=" * 70)
            print("CAMPAIGN APPROVED!")
            print("=" * 70)
            print("\nFinal Approved Campaign:")
            print("-" * 70)
            print(final_report.content)
            print("-" * 70)
        else:
            print("\n" + "=" * 70)
            print("CAMPAIGN REQUIRES REVISION")
            print("=" * 70)
            print(f"\nProceeding to iteration {iteration + 1}...")
            iteration += 1
    
    # Check if max iterations reached without approval
    if not approved:
        print("\n" + "=" * 70)
        print("MAX ITERATIONS REACHED WITHOUT APPROVAL")
        print("=" * 70)
        print("The campaign did not meet approval criteria within the iteration limit.")
    
    # Cleanup
    await runtime.stop_when_idle()
    
    print("\n" + "=" * 70)
    print("ORCHESTRATION COMPLETE")
    print("=" * 70)


 ## 7. Run the Orchestration

In [9]:
print("\nStarting Marketing Campaign Creator-Critic Orchestrator...\n")
await main()


Starting Marketing Campaign Creator-Critic Orchestrator...

MARKETING CAMPAIGN CREATOR-CRITIC ORCHESTRATION SYSTEM

ITERATION 1

----------------------------------------------------------------------
Current Task:
----------------------------------------------------------------------

Create a 5-post social media campaign for launching a new eco-friendly water bottle product.

Product Details:
- Name: HydroEco Pro
- Features: 100% recycled materials, keeps drinks cold for 24 hours, leak-proof, dishwasher safe
- Target Audience: Environmentally conscious millennials and Gen Z (ages 18-35)
- Price Point: $29.99
- Brand Voice: Friendly, inspirational, eco-focused

Campaign Goals:
- Build awareness for product launch
- Drive pre-orders
- Grow social media following
- Establish eco-friendly brand identity

----------------------------------------------------------------------

Agent Response: CampaignCreatorAgent
----------------------------------------------------------------------

Agent

 ## Key Takeaways
 
 ### 1. **Creator-Critic Pattern**
 This is a powerful quality assurance pattern:
 ```
 Creator → Draft
 Critic → Review
 If not approved → Creator revises based on feedback
 Repeat until approved
 ```
 
 **Benefits:**
-  Ensures quality output through iterative refinement
-  Separates creation from validation (single responsibility)
-  Captures institutional knowledge in critic instructions
-  Reduces human review burden
 
 ### 2. **Sequential Orchestration**
 ```python
 SequentialOrchestration(
     members=[creator_agent, critic_agent],
     agent_response_callback=callback_function
 )
 ```
 
 Key features:
 - Agents execute in **strict order**
 - Each agent sees previous agent's output
 - Callbacks provide real-time monitoring
 - Perfect for workflows with dependencies
 
 ### 3. **Iterative Refinement Loop**
 The while loop implements continuous improvement:
 ```python
 while not approved and iteration < max_iterations:
     result = await orchestration.invoke(task)
     if "APPROVED" in result:
         approved = True
     else:
         iteration += 1
         task = result  # Feedback becomes new task
 ```
 
 This creates a **feedback loop** where output improves each iteration.
 
 ### 4. **State Management**
 Three critical state variables:
 - `approved`: Boolean tracking approval status
 - `iteration`: Counter preventing infinite loops
 - `current_task`: Carries context between iterations
 
 ### 5. **InProcessRuntime**
 ```python
 runtime = InProcessRuntime()
 runtime.start()
 # ... orchestration ...
 await runtime.stop_when_idle()
 ```
 
 The runtime:
 - Manages agent lifecycle
 - Handles message passing between agents
 - Provides execution environment
 - Must be started before and stopped after use
 
 ### 6. **Callback Functions**
 ```python
 def agent_response_callback(message: ChatMessageContent) -> None:
     print(f"Agent: {message.name}")
 ```
 
 Callbacks enable:
 - Real-time monitoring
 - Logging and auditing
 - Progress tracking
 - Debugging
 
 ### 7. **When to Use This Pattern**
 
**Use Creator-Critic when:**
 - Output quality is critical
 - You have clear quality criteria
 - Iterative improvement is valuable
 - You want to encode expert review knowledge
 - Human review is expensive/slow
 
**Don't use when:**
 - First draft is usually good enough
 - Quality criteria are subjective/unclear
 - Iteration costs outweigh benefits
 - Real-time response is required
 
 ### 8. **Production Enhancements**
 
 **Limit Iterations:**
 ```python
 max_iterations = 5  # Prevent infinite loops
 timeout = 300  # 5 minute timeout
 ```
 
 **Track Iteration History:**
 ```python
 iteration_history = []
 iteration_history.append({
     "iteration": iteration,
     "draft": creator_output,
     "feedback": critic_output,
     "approved": approved
 })
 ```
 
 **Save Intermediate Results:**
 ```python
 with open(f"draft_v{iteration}.txt", "w") as f:
     f.write(final_report.content)
 ```
 
 **Add Human-in-the-Loop:**
 ```python
 if iteration > 3 and not approved:
     human_decision = input("Continue iterations? (y/n): ")
     if human_decision.lower() != 'y':
         break
 ```
 
 **Implement Scoring:**
 ```python
 critic_agent_with_scoring = ChatCompletionAgent(
     instructions="""
     Rate the campaign on a scale of 1-10 for:
     - Engagement (1-10)
     - Brand alignment (1-10)
     - CTA effectiveness (1-10)
     Average score must be ≥8 to approve.
     """
 )
 ```
 
 **Early Exit on Quality Threshold:**
 ```python
 if average_score >= 8.0:
     approved = True
 elif iteration >= max_iterations:
     print("Approving with current quality level")
     approved = True
 ```
